# Ćwiczenie 3
W niniejszym ćwiczeniu uczymy jednowarstwową sieć neuronową do klasyfikacji zwierząt na podstawie wybranych cech (np. liczba nóg, środowisko życia, zdolność latania itp.). Sieć otrzymuje na wejściu wektor cech, a na wyjściu zwraca wektor klas (np. ssak, ptak, ryba). Uczenie przebiega w trybie online – w każdej iteracji wybierany jest losowy przykład, a wagi sieci są korygowane na podstawie różnicy między oczekiwanym a rzeczywistym wyjściem.

# Cel
Celem ćwiczenia jest:
1. Zapoznanie się z budową i działaniem jednowarstwowej sieci neuronowej z funkcją aktywacji sigmoidalnej.
2. Zaimplementowanie algorytmu uczenia sieci metodą gradientu prostego (regułą delta).
3. Zbadanie, w jaki sposób dobór współczynnika uczenia i liczby epok wpływa na skuteczność klasyfikacji.

# Wstęp teoretyczny
W rozważanym modelu sieci jednowarstwowej każdemu neuronu towarzyszy funkcja aktywacji w postaci funkcji sigmoidalnej (logistycznej). Dla sygnału wejściowego $$\mathbf{p}$$ i wag $$\mathbf{W}$$ wyjście neuronu oblicza się następująco:

$$
\mathrm{net} = \mathbf{W} \cdot \mathbf{p},
$$

a następnie przeprowadza przez funkcję aktywacji:

$$
\sigma(\mathrm{net}) = \frac{1}{1 + e^{-\mathrm{net}}}.
$$

W przypadku wielu neuronów wyjściowych, każdy z nich ma własny zestaw wag, a powyższe równanie liczone jest dla każdej kolumny macierzy $$\mathbf{W}$$ (lub wiersza, w zależności od przyjętej konwencji).

Uczenie sieci polega na wielokrotnym prezentowaniu przykładów uczących – w każdej iteracji wybierany jest jeden z nich, obliczane jest wyjście sieci $$\mathbf{out}$$, a następnie błąd:

$$
\mathbf{e} = \mathbf{t} - \mathbf{out},
$$

gdzie $$\mathbf{t}$$ to wektor oczekiwany (tzw. wektor nauczyciela). Aby dostosować wagi, obliczamy wektor $$\boldsymbol{\delta}$$ z uwzględnieniem pochodnej sigmoidy:

$$
\boldsymbol{\delta} = \mathbf{e} \odot \mathbf{out} \odot \bigl(1 - \mathbf{out}\bigr),
$$

gdzie $$\odot$$ oznacza iloczyn Skalarny (element-wise). Aktualizacja wag przebiega zgodnie z regułą:

$$
\Delta \mathbf{W} = \eta \cdot \boldsymbol{\delta} \cdot \mathbf{p}^T,
$$

a następnie:

$$
\mathbf{W} \leftarrow \mathbf{W} + \Delta \mathbf{W},
$$

gdzie $$\eta$$ jest współczynnikiem uczenia (learning rate).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Definicja funkcji uczącej sieć z zapisem historii błędu
def train_single_layer_with_error(P, T, W, n, eta=0.1):
    """
    Funkcja ucząca jednowarstwową sieć neuronową z aktywacją sigmoidalną
    oraz zapisywaniem błędu średniokwadratowego (MSE) w każdej epoce.

    Parametry:
      P  : macierz wejść (liczba_cech x liczba_przykładów)
      T  : macierz wyjść oczekiwanych (liczba_wyjść x liczba_przykładów)
      W  : macierz wag (liczba_wyjść x liczba_cech)
      n  : liczba epok (iteracji przez zbiór uczący)
      eta: współczynnik uczenia

    Zwraca:
      W : zaktualizowaną macierz wag
      errors : lista wartości MSE obliczanych na koniec każdej epoki
    """
    num_samples = P.shape[1]
    errors = []

    for epoch in range(n):
        epoch_errors = []
        # Losowa kolejność przykładów w każdej epoce
        indices = np.random.permutation(num_samples)
        for i in indices:
            # Pobieramy przykładowe wejście i wektor nauczyciela (kolumnowo)
            p = P[:, i].reshape(-1, 1)
            t = T[:, i].reshape(-1, 1)

            # Propagacja w przód: obliczenie net i aktywacji sigmoidalnej
            net = np.dot(W, p)
            out = 1 / (1 + np.exp(-net))

            # Obliczenie błędu
            e = t - out

            # Delta – błąd uwzględniający pochodną funkcji sigmoidalnej
            delta = e * out * (1 - out)

            # Aktualizacja wag
            W += eta * np.dot(delta, p.T)

            # Zapis błędu dla tego przykładu
            epoch_errors.append(np.mean(e**2))

        # Średni błąd epoki
        errors.append(np.mean(epoch_errors))

    return W, errors

In [ ]:
# Przykładowa definicja macierzy wejść P (każda kolumna to inny przykład)
# Poniżej 5 przykładowych kolumn (zwierząt), a 4 cechy w wierszach.
P = np.array([
    [4.0,  0.0,  1.0,  4.0,  1.0],  # liczba nóg
    [1.0,  1.0,  0.0,  1.0,  0.0],  # czy żyje w wodzie
    [0.0,  1.0,  1.0,  0.0,  1.0],  # czy potrafi latać
    [1.0,  3.5,  0.0,  2.0,  0.0],  # przykładowa inna cecha
], dtype=float)

# Przykładowa definicja macierzy nauczyciela T (każda kolumna odpowiada tej samej kolumnie w P)
# Załóżmy, że mamy 3 klasy: [ssak, ptak, ryba].
T = np.array([
    [1, 0, 0, 1, 0],  # ssak
    [0, 1, 0, 0, 1],  # ptak
    [0, 0, 1, 0, 0],  # ryba
], dtype=float)

# Inicjalizacja wag (losowo), wymiary: (liczba_wyjść x liczba_wejść)
# Tutaj liczba_wyjść = 3, liczba_wejść = 4.
W_poczatkowe = np.random.randn(T.shape[0], P.shape[0])

# Parametry uczenia
n = 10       # liczba epok
eta = 0.1    # współczynnik uczenia

# Funkcja 'train_single_layer' powinna być zaimportowana lub zdefiniowana w tym samym pliku/skrypcie
W_nauczone = train_single_layer(P, T, W_poczatkowe, n, eta)

print("Wyuczone wagi:\n", W_nauczone)


In [ ]:
W_nauczone = train_single_layer(P, T, W_poczatkowe, n=10, eta=0.1)

In [ ]:
p_new = np.array([[4], [1], [0], [1]])  # teraz ma kształt (4,1)
net_new = np.dot(W_nauczone, p_new)
out_new = 1 / (1 + np.exp(-net_new))
print("Wyjście sieci dla nowego przykładu:", out_new)

In [ ]:
# Parametry uczenia
n = 10       # liczba epok
eta = 0.1    # współczynnik uczenia

# Trening sieci oraz zapis historii błędu
W_nauczone, error_history = train_single_layer_with_error(P, T, W_poczatkowe, n, eta)

print("Wyuczone wagi:\n", W_nauczone)

In [ ]:
sns.set(style="whitegrid")

# Wykres MSE w kolejnych epokach
plt.figure(figsize=(8, 5))
sns.lineplot(x=np.arange(1, n+1), y=error_history, marker="o")
plt.title("Wykres błędu średniokwadratowego (MSE) w kolejnych epokach")
plt.xlabel("Epoka")
plt.ylabel("Błąd średniokwadratowy (MSE)")
plt.show()

# Heatmapa macierzy wag nauczonej
plt.figure(figsize=(6, 4))
ax = sns.heatmap(W_nauczone, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Macierz wag nauczonej sieci")
plt.xlabel("Wejścia (cechy)")
plt.ylabel("Neurony wyjściowe (klasy)")
plt.show()

# Wnioski
1. Sieć jednowarstwowa z funkcją sigmoidalną może skutecznie rozróżniać przykłady należące do różnych klas, jeśli dane są odpowiednio dobrane i wystarczająco zróżnicowane.
2. Zbyt mała wartość
    $$\eta$$
   powoduje wolną zbieżność algorytmu, natomiast zbyt duża może prowadzić do oscylacji i utrudnić osiągnięcie minimum błędu.
3. Liczba epok treningowych powinna być dobrana tak, aby sieć mogła wystarczająco często zobaczyć każdy przykład i dostosować do niego wagi.
4. Dodanie wiersza jedynek (bias) do macierzy wejściowej może znacząco poprawić skuteczność klasyfikacji, ponieważ pozwala sieci na przesunięcie hiperpowierzchni decyzyjnej w przestrzeni cech.
5. Podejście online, z losowym doborem przykładów, może zapobiec lokalnym minimom i przyspieszyć uczenie w porównaniu do metody batch.